In [1]:
!pip install transformers datasets evaluate rouge_score accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=faa54ea8aeec036fb7a248d24d70cbf20318ebb27bb79aea4da4f0f95fd26edd
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependenc

In [2]:
import transformers

transformers.__version__

'4.44.2'

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from huggingface_hub import list_datasets

datasets_list = list_datasets()

num = 0
for dataset in datasets_list:
  print(dataset)

  if (num > 10):
    break

  num += 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetInfo(id='amirveyseh/acronym_identification', author='amirveyseh', sha='15ef643450d589d5883e289ffadeb03563e80a9e', created_at=datetime.datetime(2022, 3, 2, 23, 29, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 1, 9, 11, 39, 57, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=925, downloads_all_time=None, likes=19, paperswithcode_id='acronym-identification', tags=['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'source_datasets:original', 'language:en', 'license:mit', 'size_categories:10K<n<100K', 'format:parquet', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'arxiv:2010.14678', 'region:us', 'acronym-identification'], card_data=None, siblings=None)
DatasetInfo(id='ade-benchmark-corpus/ade_corpus_v2', author='ade-benchmark-corpus', sha='4ba01c71687dd7c996597042449448ea312126cf', 

In [6]:
from datasets import load_dataset

ds = load_dataset("microsoft/MeetingBank-QA-Summary")
ds

README.md:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/7.29M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/862 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['idx', 'prompt', 'QA_pairs', 'summary', 'gpt4_summary'],
        num_rows: 862
    })
})

In [7]:
ds.shape

{'test': (862, 5)}

In [10]:
ds['test'].features

{'idx': Value(dtype='int64', id=None),
 'prompt': Value(dtype='string', id=None),
 'QA_pairs': [{'answer': Value(dtype='string', id=None),
   'question': Value(dtype='string', id=None)}],
 'summary': Value(dtype='string', id=None),
 'gpt4_summary': Value(dtype='string', id=None)}

In [11]:
ds.homepage

AttributeError: 'DatasetDict' object has no attribute 'homepage'

In [12]:
ds_info = ds['test'].info
homepage = ds_info.homepage if hasattr(ds_info, 'homepage') else "No homepage available"
print(homepage)

In [14]:
ds['test'][0]

{'idx': 0,
 'prompt': "The report of the Civil Rights, Utilities, Economic Development and Arts Committee Agenda Item three Resolution 31669 Encouraging as a best practice the use of an individualized tenant assessment using the Fair Housing Act's discriminatory effect standards to avoid Fair Housing Act violations when criminal history is used as a screening criterion in the Landlord Screening Process, Committee recommends that the resolution be adopted as amended grade. I move to amend Resolution 31669 by substituting D four for version D three, which includes a new attachment. A And I understand Councilmember Bagshaw also has an amendment, but let's first, if we could, let me just go through the changes to the resolution since the last committee meeting. The changes are found in two recitals, as well as sections one and five are the primary changes. We added a recital that again lifts up the HUD guidance to show that a criminal history screening policy is next must serve a substanti

In [20]:
def clean_txt(example):
    for txt in ["summary", "gpt4_summary"]:
        example[txt] = example[txt].lower()
        example[txt] = example[txt].replace("\\", "")  # Corrected for backslashes
        example[txt] = example[txt].replace("/", "")
        example[txt] = example[txt].replace("\n", "")
        example[txt] = example[txt].replace("``", "")
        example[txt] = example[txt].replace('"', '')
        example[txt] = example[txt].replace("--", "")

    return example


In [21]:
cleaned_ds = ds.map(clean_txt)
cleaned_ds

Map:   0%|          | 0/862 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['idx', 'prompt', 'QA_pairs', 'summary', 'gpt4_summary'],
        num_rows: 862
    })
})

In [24]:
# Access the 7th element from the 'summary' field in the 'test' split
ds['test']['summary'][7]


'Recommendation to authorize City Manager, or designee, to execute all necessary documents to amend the U.S. Economic Development Administration (EDA) Revolving Loan Fund, to create the Long Beach Emergency Microenterprise Loan Program to assist local micro businesses impacted by the COVID-19 pandemic during the emergency declaration period.  (Citywide)'

In [25]:
cleaned_ds['test']['summary'][7]

'recommendation to authorize city manager, or designee, to execute all necessary documents to amend the u.s. economic development administration (eda) revolving loan fund, to create the long beach emergency microenterprise loan program to assist local micro businesses impacted by the covid-19 pandemic during the emergency declaration period.  (citywide)'

In [28]:
ds['test']['gpt4_summary'][0]


"The Civil Rights, Utilities, Economic Development and Arts Committee discussed Resolution 31669, which encourages the use of an individualized tenant assessment using the Fair Housing Act's discriminatory effect standards to avoid violations when criminal history is used as a screening criterion in the Landlord Screening Process. The committee recommended adopting the resolution with amendments. The changes include adding a recital that a criminal history screening policy must serve a substantial, legitimate and nondiscriminatory interest, referencing the Seattle Fair Chance Employment Ordinance, and focusing on the impacts of the resolution on future housing screening practices. The resolution also encourages landlords to follow HUD guidance regarding criminal records and endorses practices that landlords should not automatically exclude individuals for housing based on prior arrests. Councilmember Bagshaw proposed an amendment to recognize the Certificate of Restoration of Opportuni

In [29]:
cleaned_ds['test']['gpt4_summary'][0]

"the civil rights, utilities, economic development and arts committee discussed resolution 31669, which encourages the use of an individualized tenant assessment using the fair housing act's discriminatory effect standards to avoid violations when criminal history is used as a screening criterion in the landlord screening process. the committee recommended adopting the resolution with amendments. the changes include adding a recital that a criminal history screening policy must serve a substantial, legitimate and nondiscriminatory interest, referencing the seattle fair chance employment ordinance, and focusing on the impacts of the resolution on future housing screening practices. the resolution also encourages landlords to follow hud guidance regarding criminal records and endorses practices that landlords should not automatically exclude individuals for housing based on prior arrests. councilmember bagshaw proposed an amendment to recognize the certificate of restoration of opportuni

In [30]:
EXAMPLE_TEXT_INDEX = 33

example_text = ds["test"]["summary"][EXAMPLE_TEXT_INDEX]

example_text

'AN ORDINANCE relating to land use regulation of home occupations; extending for six months interim development controls established by Ordinance 126293 to allow home-based businesses to operate with fewer limitations during the COVID-19 civil emergency; and ratifying and confirming certain prior acts.'

In [31]:
MODEL_NAME = "t5-small"

In [32]:

from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline("summarization", model = MODEL_NAME)

summary_txt = summarizer(prefix + example_text)

summary_txt

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 200, but your input_length is only 69. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


[{'summary_text': 'extending for six months interim development controls established by Ordinance 126293 to allow home-based businesses to operate with less limitations during the COVID-19 civil emergency . ratifying and confirming certain prior acts .'}]

In [33]:
summarizer = pipeline("summarization", model = MODEL_NAME, truncation=True)


summary_txt = summarizer(prefix + example_text)

summary_txt

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 200, but your input_length is only 69. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


[{'summary_text': 'extending for six months interim development controls established by Ordinance 126293 to allow home-based businesses to operate with less limitations during the COVID-19 civil emergency . ratifying and confirming certain prior acts .'}]

In [34]:
ref_txt = ds["test"]["gpt4_summary"][EXAMPLE_TEXT_INDEX]

ref_txt

'The meeting discussed two main items. The first was Council Bill 12265, which extends the policies for home-based businesses adopted last year under the Bringing Business Home legislation. This extension will allow departments to work on permanent regulations. The changes made it easier to operate a home-based business during the pandemic by easing restrictions, including removing the limit of two non-resident employees, removing the restriction that customers can only visit by appointment, and ending the requirement that businesses retain the appearance of a home. The bill passed with six in favor and one opposed. \n\nThe second item was Resolution 32048, which approves and ratifies the decision of the Metropolitan King County Council to adopt a revised set of countywide planning policies. The committee recommended the resolution be adopted.'

In [35]:
import evaluate

rouge = evaluate.load("rouge")

rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [36]:
reference = "the elephant was found near the river and everyone was glad"

ex_1 = "the big elephant was found near the turbulent river and everyone was ecstatic"

ex_2 = "the big elephant was found near the river and made everyone glad"

In [37]:
results = rouge.compute(predictions = [ex_1], references = [reference])

results

{'rouge1': 0.8333333333333333,
 'rouge2': 0.6363636363636365,
 'rougeL': 0.8333333333333333,
 'rougeLsum': 0.8333333333333333}

In [38]:

results = rouge.compute(predictions = [ex_2], references = [reference])

results

{'rouge1': 0.8695652173913043,
 'rouge2': 0.5714285714285713,
 'rougeL': 0.8695652173913043,
 'rougeLsum': 0.8695652173913043}

In [39]:

summary_txt[0]["summary_text"]

'extending for six months interim development controls established by Ordinance 126293 to allow home-based businesses to operate with less limitations during the COVID-19 civil emergency . ratifying and confirming certain prior acts .'

In [40]:
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = True)
result

{'rouge1': 0.2073170731707317,
 'rouge2': 0.04938271604938272,
 'rougeL': 0.12195121951219512,
 'rougeLsum': 0.12195121951219512}

In [41]:

article_texts = ds["test"]["summary"]

article_summaries = ds["test"]["gpt4_summary"]

In [42]:

from tqdm import tqdm

candidate_summaries = []

prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(prefix + text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [01:42<00:00,  2.05s/it]


In [43]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.22169840170513566,
 'rouge2': 0.11409453814098913,
 'rougeL': 0.17047368394883355,
 'rougeLsum': 0.171529652178102}

In [44]:
result_unagg = rouge.compute(predictions = candidate_summaries,
                             references = article_summaries[:50],
                             use_stemmer = True,
                             use_aggregator = False)
result_unagg

{'rouge1': [0.23913043478260868,
  0.26993865030674846,
  0.22448979591836735,
  0.09473684210526316,
  0.23841059602649006,
  0.24050632911392403,
  0.16666666666666669,
  0.3134328358208955,
  0.2738095238095238,
  0.15942028985507245,
  0.41573033707865165,
  0.2162162162162162,
  0.2608695652173913,
  0.14634146341463414,
  0.11320754716981132,
  0.2368421052631579,
  0.1839080459770115,
  0.32116788321167883,
  0.22916666666666663,
  0.2459893048128342,
  0.3185840707964602,
  0.25698324022346364,
  0.09137055837563451,
  0.26285714285714284,
  0.1595744680851064,
  0.2857142857142857,
  0.21568627450980393,
  0.12658227848101267,
  0.2112676056338028,
  0.16,
  0.25925925925925924,
  0.13658536585365855,
  0.2896551724137931,
  0.2073170731707317,
  0.27142857142857146,
  0.15384615384615385,
  0.22891566265060243,
  0.12578616352201258,
  0.32298136645962733,
  0.27218934911242604,
  0.18556701030927836,
  0.25287356321839083,
  0.36496350364963503,
  0.05970149253731343,
  0.29

In [45]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(42, 13)

In [46]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,a RESOLUTION encouraging as a best practice th...,"The Civil Rights, Utilities, Economic Developm..."
1,this authorization applies to contracts for go...,The Committee on Government Operations discuss...
2,resolution appoints certain properties within ...,The City Manager recommended the adoption of t...
3,a new concurrency test results map has been ad...,The meeting discussed proposed ordinance 2016 ...
4,a ORDINANCE relating to the department of park...,The meeting discussed the confirmation of an a...


In [47]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(38, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(38, "Reference_summaries"))

Actual Summary
a MOTION authorizes the council to enter into a contract for services to represent the county with the Washington state Legislature . the council chair will be appointed by the council in june .
Reference Summary
The council meeting discussed a proposed motion (0264) to authorize the council chair to enter into a contract for services to represent the county before the Washington State Legislature. Jeff Mumm from council staff provided a report, mentioning that an RFP was issued earlier this year, and Amendment One would insert foster government relations as the RFP awardee. The contract is for two years, with an option for a one-year extension. Councilmember Bell made a motion for approval, which was passed. The contract amount is the same as the previous one, at $11,000 per month. The council also discussed motion 2021 to 89, which requests an assessment report on roles and responsibilities of the county, city of Seattle, and King County Regional Homelessness Authority

In [48]:

print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(12, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(12, "Reference_summaries"))

Actual Summary
recommendation to adopt resolution approving the Proposed Fiscal Year 2016 Harbor Department Salary Resolution No. HD-2822.
Reference Summary
The meeting involved discussions on various topics, with some participants expressing their thoughts and feelings. The main point of the meeting was the discussion and voting on Item number 29, which was a report from the Harbor Department. The report recommended the adoption of a resolution approving the proposed fiscal year 2016 Harbor Department Salary Resolution. There was no presentation for this item, but a motion was made and seconded. No public comments were made on the resolution, and the motion was carried. The meeting then proceeded to the next agenda item, which was hearing item number two.


In [49]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [50]:
tokenizer(["four-time defending champion"])

{'input_ids': [[662, 18, 715, 3, 20309, 6336, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [51]:
tokens = tokenizer.tokenize("four-time defending champion")

tokens

['▁four', '-', 'time', '▁', 'defending', '▁champion']

In [52]:
ids = tokenizer.convert_tokens_to_ids(tokens)

ids

[662, 18, 715, 3, 20309, 6336]

In [53]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [54]:
prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["summary"]]
    model_inputs = tokenizer(inputs, max_length = 1024, truncation = True)

    labels = tokenizer(text_target = examples["gpt4_summary"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [55]:
tokenized_ds = cleaned_ds.map(preprocess_function, batched = True)

tokenized_ds

Map:   0%|          | 0/862 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['idx', 'prompt', 'QA_pairs', 'summary', 'gpt4_summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 862
    })
})

In [56]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [57]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}

In [58]:
from huggingface_hub import notebook_login

notebook_login()

In [71]:
training_args = Seq2SeqTrainingArguments(
    output_dir="meetingbank_qa_summary_model_finetuned_on_reduced_data",  # Removed trailing space
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['test'],  # Using 'test' for training as well
    eval_dataset=tokenized_ds['test'],    # Using 'test' for evaluation
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Generated Length
1,No log,3.165680,0.181800,0.088600,0.153300,0.153200,19.000000
2,No log,3.135147,0.184700,0.090900,0.156200,0.156100,19.000000
3,No log,3.126129,0.185800,0.091100,0.157300,0.157100,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=162, training_loss=3.4018385145399304, metrics={'train_runtime': 110.469, 'train_samples_per_second': 23.409, 'train_steps_per_second': 1.466, 'total_flos': 154450900746240.0, 'train_loss': 3.4018385145399304, 'epoch': 3.0})

In [72]:
trainer.push_to_hub()

events.out.tfevents.1727283523.099b5b81c40c.1322.2:   0%|          | 0.00/7.96k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ZaidZia/meetingbank_qa_summary_model_finetuned_on_reduced_data/commit/b6f6667a5aaf62d4787b7bfb1649c3fdd4e9a44d', commit_message='End of training', commit_description='', oid='b6f6667a5aaf62d4787b7bfb1649c3fdd4e9a44d', pr_url=None, pr_revision=None, pr_num=None)

In [73]:

example_text

'AN ORDINANCE relating to land use regulation of home occupations; extending for six months interim development controls established by Ordinance 126293 to allow home-based businesses to operate with fewer limitations during the COVID-19 civil emergency; and ratifying and confirming certain prior acts.'

In [74]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline(
    "summarization",
    model = "ZaidZia/meetingbank_qa_summary_model_finetuned_on_reduced_data",
    truncation=True
)
summary_txt = summarizer(prefix + example_text)

summary_txt

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 200, but your input_length is only 69. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


[{'summary_text': 'a new ORDINANCE relating to land use regulation of home occupations, extending for six months interim development controls established by Ordinance 126293 to allow home-based businesses to operate with less limitations during the COVID-19 civil emergency, and ratifying and confirming certain prior acts. the ORDinANCE relates to the land use regulations and the development of the home occupation. it also includes extending the development controls for fewer restrictions during the civil emergency and confirming certain prior actes. the legislation was adopted by a senate, which includes a six-month period to allow homes to operate in the area for the first time in april to allow for the development to be a part of the .'}]

In [75]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ZaidZia/meetingbank_qa_summary_model_finetuned_on_reduced_data")

inputs = tokenizer(prefix + example_text, return_tensors = "pt", truncation=True).input_ids

In [77]:
model = AutoModelForSeq2SeqLM.from_pretrained("ZaidZia/meetingbank_qa_summary_model_finetuned_on_reduced_data")

outputs = model.generate(inputs, max_new_tokens = 100, do_sample = False)

tokenizer.decode(outputs[0], skip_special_tokens = True)

'the ORDINANCE relates to land use regulation of home occupations and the extension for six months interim development controls established by Ordinance 126293 to allow home-based businesses to operate with less limitations during the COVID-19 civil emergency. the ORDINANCE also relates to the extending for six months interim development controls established by Ordinance 126293 to allow home-based businesses to operate with less limitations during the COVID-19 civil emergency. the ORDIN'

In [78]:

ref_txt

'The meeting discussed two main items. The first was Council Bill 12265, which extends the policies for home-based businesses adopted last year under the Bringing Business Home legislation. This extension will allow departments to work on permanent regulations. The changes made it easier to operate a home-based business during the pandemic by easing restrictions, including removing the limit of two non-resident employees, removing the restriction that customers can only visit by appointment, and ending the requirement that businesses retain the appearance of a home. The bill passed with six in favor and one opposed. \n\nThe second item was Resolution 32048, which approves and ratifies the decision of the Metropolitan King County Council to adopt a revised set of countywide planning policies. The committee recommended the resolution be adopted.'

In [79]:

result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = False)
result

{'rouge1': 0.3790322580645162,
 'rouge2': 0.04878048780487806,
 'rougeL': 0.20161290322580647,
 'rougeLsum': 0.217741935483871}

In [80]:
candidate_summaries = []

for i, text in enumerate(tqdm(article_texts[:50])):

  candidate = summarizer(prefix + text)

  candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [03:56<00:00,  4.73s/it]


In [81]:

result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:50], use_stemmer = False)
result_agg

{'rouge1': 0.35360948508916035,
 'rouge2': 0.14227490511611163,
 'rougeL': 0.23320533686503994,
 'rougeLsum': 0.23589984593949526}

In [82]:
result_unagg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                             use_stemmer = True, use_aggregator = False)

In [83]:

result_unagg_rsum = np.array(result_unagg["rougeLsum"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)


(5, 43)

In [84]:

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,a RESOLUTION encourages the use of an individu...,"The Civil Rights, Utilities, Economic Developm..."
1,this authorization applies to contracts for go...,The Committee on Government Operations discuss...
2,the amendment to a certain joint powers agreem...,The City Manager recommended the adoption of t...
3,a new map indicating the boundaries of the con...,The meeting discussed proposed ordinance 2016 ...
4,"the deed was approved for open space, park, an...",The meeting discussed the confirmation of an a...


In [85]:

print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(44, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(44, "Reference_summaries"))


Actual Summary
the meeting discussed an open space improvements over and across Utah Avenue South between South Stacy Street and South Lander Street. the meeting was held on thursday, a day after the meeting, the meeting held a meeting with the public about the project. the project included a proposal to amend the ordinance 121572, updating the insurance and bond requirements, and renewing the term of the permit to First & Utah Street Associates, LLC, providing for the acceptance of the permits and conditions. the proposal was approved by the public and the public. the agreement was a reopen space between south and the intersection of the . the two . it was the first to discuss the issue of the open space between .
Reference Summary
The Transportation Committee discussed Council Bill 118482, which pertains to open space improvements over Utah Avenue South. The bill proposes a ten-year renewal of a permit to maintain a pedestrian plaza in front of the former Sears building on South Utah

In [86]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(23, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(23, "Reference_summaries"))

Actual Summary
the zoning classification was changed from B-A-3 (Former code, large-lot commercial) to C-MX-3 (Urban Center context, mixed use, 3 and 12 stories maximum) for property at 1465 S. Colorado Boulevard in council district 6. a PUBLIC HEARING WILL BE HELD AT LEAST FOUR WEEKS. the committee approved filing this bill at its meeting on 1-28-14.
Reference Summary
The public hearing for council Bill 35 discussed a proposed rezoning at 1465 South Colorado Boulevard. The rezoning is from B.A. three to CMCs three and CMCs 12, which are mixed-use three-story and 12-story zone districts. The site is currently underutilized, with a large parking lot and a Courtyard by Marriott Hotel. The rezoning aims to bring the site into conformance with the current Denver zoning code and to make better use of the space. The proposal includes small commercial uses in front of the hotel facing Colorado Boulevard. The plan received unanimous approval from the Planning Board and support from the Corey M